# US

STATA에서 step01과 step03을 거친이후에 가능

In [35]:
using Plots, CSV, DataFrames, XLSX

In [38]:
# 데이터 로드
OMPS = CSV.read("OMPS.csv", DataFrame)
OMPS = OMPS[:, 2]

HP_data = CSV.read("Zillow.csv", DataFrame, header=true)
CPI_data = XLSX.readdata("BLS_CPI_MSA.xlsx", "CPI", "B3:H293");

291×7 Matrix{Any}:
 100      100      100      100      100      100      100
 100.795  100.606  100.427  100.585  100.669  100.705  100.834
 101.305  101.334  101.463  101.462  101.227  101.175  101.668
 101.305  101.273  101.584  101.579  101.171  100.999  101.608
 101.249  101.577  101.584  101.754  101.171  102.056  101.906
 101.589  102.911  102.072  101.93   101.506  103.408  101.846
 102.043  102.365  102.377  102.456  101.952  102.585  102.263
 102.1    102.001  102.377  102.865  102.119  102.056  102.561
 102.554  103.093  102.681  103.275  102.844  102.703  103.216
 102.838  103.153  102.681  103.626  102.956  103.055  103.514
 103.008  103.275  102.742  103.626  102.956  103.408  103.335
 102.894  103.214  102.62   103.567  102.733  103.29   103.335
 103.405  104.245  103.169  104.269  103.123  104.642  103.752
   ⋮                                            ⋮      
 176.985  170.968  179.96   188.413  178.032  165.936  190.643
 177.208  171.411  180.31   189.196  178.473  1

In [64]:
# metro 별 분석

filtered_data = HP_data[HP_data.metro_id .== 17, :]

# Local Projection
# 매개변수 설정
H = 40
M = 12
I = 2
MPS_start = 157;

# 2001m2~2019m12
S = OMPS[MPS_start:end] 
N_s = length(S)
# 2000m1~2024m2
metro_data = filtered_data[:,4]
N = 291
metro_data = reshape(metro_data, N,:)
~, K = size(metro_data)

# deflate
def_data = zeros(N,K)
for k=1:K
    def_data[:,k] = log.(metro_data[:,k]).-log.(CPI_data[:,4])
end


start_idx = 14; # 2001m2

# shock variables
if I!=0
    S_lag = zeros(N_s,I)
    Threads.@threads for i=1:I
        S_lag[:,i]= OMPS[MPS_start-i:end-i]
    end
end

b_h = zeros(H+1, K);
Threads.@threads for k=1:K
    Y_lag = zeros(N_s,M)
    for m=1:M
        Y_lag[:,m]=def_data[start_idx-m:(start_idx+N_s-1)-m,k] - def_data[start_idx-m-1:(start_idx+N_s-1)-m-1,k]
    end
    if I!=0
        X = [S S_lag Y_lag ones(N_s,1)]
    else
        X = [S Y_lag ones(N_s,1)]
    end
    sample_num, explain_num = size(X)
    df = sample_num-explain_num
    
    Threads.@threads for h=0:H
        
        # regression
        dep = def_data[start_idx+h:(start_idx+N_s-1)+h,k]-def_data[start_idx-1:(start_idx+N_s-1)-1,k]
        beta = (X'*X) \ (X'*dep)
        e = dep - X*beta
        
        # save
        b_h[h+1,k] = beta[1]
    end
end